# COVID-19 WHO-HealthPropaPhenKG Evaluation

In [1]:
%load_ext autoreload
%autoreload 2

## Introduction

### Standard

In [2]:
import numpy as np
import pandas as pd

### Custom

## Paths

Download the following files:  
url = "https://covid19.who.int/WHO-COVID-19-global-data.csv"  
url_iso = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/UID_ISO_FIPS_LookUp_Table.csv"

In [3]:
path_who_data = "../data/groudtruth/WHO-COVID-19-global-data.csv"
path_isodata = "../data/groudtruth/UID_ISO_FIPS_LookUp_Table.csv"
path_coviddata = "../data/groudtruth/covidToWorld.csv"

In [4]:
path_to_netwoork_gazetteer = '../data/gazetteers/world_gazetteer_en.csv'

## Reading WHO data

In [5]:
data = pd.read_csv(path_who_data)

In [6]:
data = data[data["Cumulative_cases"] > 0]

In [7]:
data.loc[data["Country"] == 'Viet Nam',"Country"] = "Vietnam"

In [8]:
data

,Date_reported,Country_code,Country,WHO_region,New_cases,Cumulative_cases,New_deaths,Cumulative_deaths
8,2020-03-01,AF,Afghanistan,EMRO,1.0,1,NaN,0
9,2020-03-08,AF,Afghanistan,EMRO,NaN,1,NaN,0
10,2020-03-15,AF,Afghanistan,EMRO,6.0,7,NaN,0
11,2020-03-22,AF,Afghanistan,EMRO,17.0,24,NaN,0
12,2020-03-29,AF,Afghanistan,EMRO,67.0,91,2.0,2
...,...,...,...,...,...,...,...,...
52315,2024-02-04,ZW,Zimbabwe,AFRO,31.0,266319,NaN,5737
52316,2024-02-11,ZW,Zimbabwe,AFRO,9.0,266328,NaN,5737
52317,2024-02-18,ZW,Zimbabwe,AFRO,22.0,266350,NaN,5737
52318,2024-02-25,ZW,Zimbabwe,AFRO,5.0,266355,2.0,5739


In [9]:
df_network = pd.read_csv(path_to_netwoork_gazetteer)

In [10]:
df_network['Name'] = df_network['Name'].apply(lambda x: x.replace('"','').lower())

In [11]:
data["Country"] = data["Country"].apply(lambda x: x.replace('"','').lower())

In [12]:
df_network = df_network.drop(columns=['Unnamed: 0'])
data = data.drop(['Country_code', 'WHO_region', 'New_cases', 'New_deaths',
       'Cumulative_deaths'],axis=1)

In [13]:
data

,Date_reported,Country,Cumulative_cases
8,2020-03-01,afghanistan,1
9,2020-03-08,afghanistan,1
10,2020-03-15,afghanistan,7
11,2020-03-22,afghanistan,24
12,2020-03-29,afghanistan,91
...,...,...,...
52315,2024-02-04,zimbabwe,266319
52316,2024-02-11,zimbabwe,266328
52317,2024-02-18,zimbabwe,266350
52318,2024-02-25,zimbabwe,266355


In [14]:
maindata = pd.merge(data,df_network,left_on="Country",right_on="Name")
maindata = maindata.drop(['Name', 'Country'],axis=1)

In [15]:
maindata['CUI:START_ID']= maindata.shape[0]*['C5203670']
maindata[':TYPE']= maindata.shape[0]*["isReported"]
maindata = maindata.rename(columns={"ID": "ID:END_ID"})

In [16]:
maindata.to_csv(path_coviddata)